In [1]:
%%writefile Server19.py
import socket
import ssl
import os
import logging


def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('localhost', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = find_free_port()
print("Free port number:", port)


SERVER_CERT = 'server.crt'  # New server certificate file
SERVER_KEY = 'server.key'    # New server private key file
SERVER_ROOT = 'C:\\Users\\shruj'  # Directory where files are stored

logging.basicConfig(filename='server.log', level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

def authenticate_user(username, password):
    # Dummy authentication function, replace with your own authentication mechanism
    return username == 'admin' and password == 'password123'

def serve_client(connection, address):
    logging.info(f"Connection established with {address}")
    try:
        while True:
            data = connection.recv(1024).decode()
            if not data:
                break
            
            command, *args = data.split()
            if command == 'GET':
                filename = args[0]
                if len(args) == 2:
                    username, password = args[1].split(':')
                    if authenticate_user(username, password):
                        file_path = os.path.join(SERVER_ROOT,filename)
                        if os.path.isfile(file_path):
                            with open(file_path, 'rb') as file:
                                while True:
                                    chunk = file.read(1024)
                                    if not chunk:
                                        break
                                    connection.sendall(chunk)
                            logging.info(f"Sent file '{filename}' to {address}")
                        else:
                            connection.sendall(b'File not found')
                            logging.warning(f"File '{filename}' not found for user {username}")
                    else:
                        connection.sendall(b'Authentication failed')
                        logging.warning(f"Authentication failed for user {username}")
                else:
                    connection.sendall(b'Missing username/password')
                    logging.warning("Missing username/password")
            else:
                connection.sendall(b'Invalid command')
                logging.warning(f"Invalid command from {address}: {command}")

    finally:
        connection.close()

def main():
    context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
    context.load_cert_chain(certfile=os.path.join(os.path.dirname(__file__), SERVER_CERT), keyfile=os.path.join(os.path.dirname(__file__), SERVER_KEY))

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind(('localhost', port))  # Use the dynamically found port
        server_socket.listen(5)

        with context.wrap_socket(server_socket, server_side=True) as ssl_socket:
            logging.info("Server listening on port %d..." % port)  # Update logging
            while True:
                connection, address = ssl_socket.accept()
                serve_client(connection, address)  # Pass the SSL socket to serve_client

if __name__ == "__main__":
    main()


Overwriting Server19.py
